# Building RAG Chatbots for Technical Documentation

## Table of contents

- [Introduction](#introduction)
- [Environment Setup](#environment-setup)
- [Load and split the document](#load-and-split-the-document)
- [Generate and store the embeddings](#generate-and-store-the-embeddings)

## Introduction 

This project involves implementing a retrieval augmented generation (RAG) with `LangChain` to create a chatbot for
answering questions about technical documentation. The document chosen for this assignment was the following: The European Union Medical Device Regulation - Regulation (EU) 2017/745 (EU MDR). 

## Environment Setup

Install the packages and dependencies to be used:

In [30]:
# Install required libraries
%pip install -qU langchain langchain-community langchain-chroma langchain-text-splitters unstructured sentence_transformers langchain-huggingface huggingface_hub pdfplumber langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


## Load and split the document

In [31]:
# Using PDF document

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("document.pdf")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
pages = loader.load_and_split(text_splitter)

print(pages[0])


page_content='02017R0745 — EN — 09.07.2024 — 004.001 — 1
This text is meant purely as a documentation tool and has no legal effect. The Union's institutions do not assume any liability
for its contents. The authentic versions of the relevant acts, including their preambles, are those published in the Official
Journal of the European Union and available in EUR-Lex. Those official texts are directly accessible through the links
embedded in this document
►B REGULATION (EU) 2017/745 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL
of 5 April 2017
on medical devices, amending Directive 2001/83/EC, Regulation (EC) No 178/2002 and
Regulation (EC) No 1223/2009 and repealing Council Directives 90/385/EEC and 93/42/EEC
(Text with EEA relevance)
(OJ L 117, 5.5.2017, p. 1)
Amended by:
Official Journal
No page date
►M1 Regulation (EU) 2020/561 of the European Parliament and of the L 130 18 24.4.2020
Council of 23 April 2020
►M2 Commission Delegated Regulation (EU) 2023/502 of 1 December 2022 L 70 1 8.

## Generate and store the embeddings

In [32]:
# Generate and store the embeddings
from langchain_chroma import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# vectorstore = Chroma.from_documents(documents=pages, embedding=embeddings, persist_directory="db")
vectorstore = Chroma(persist_directory="db", embedding_function=embeddings)

 ## Retrieve


In [33]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("Describe the use of harmonised standards")

for doc in retrieved_docs:
    print("page " + str(doc.metadata["page"] + 1) + ":", doc.page_content[:300])

page 16: which have been published in the Official Journal of the European
Union, shall be presumed to be in conformity with the requirements
of this Regulation covered by those standards or parts thereof.
The first subparagraph shall also apply to system or process
requirements to be fulfilled in accordance
page 197: used, particularly as regards sterilisation and the relevant documents;
and
(e) the appropriate tests and trials which are to be carried out before,
during and after manufacture, the frequency with which they are to
take place, and the test equipment to be used; it shall be possible to
trace back ad
page 168: an initial start-up phase.
1.6. Participation in coordination activities
1.6.1. The notified body shall participate in, or ensure that its assessment
personnel is informed of, any relevant standardisation activities and in
the activities of the notified body coordination group referred to in
Article


## LLM

In [34]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import pipeline, set_seed

generator = pipeline('text-generation', model='gpt2', max_length=1000, pad_token_id=50256, return_full_text=False)

gpt2 = HuggingFacePipeline(pipeline=generator)

set_seed(42)
generator("Describe the use of harmonised standards", max_length=30, num_return_sequences=5)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': ':\n\nThis is an extremely important question—how can a large proportion of people who may not speak the'},
 {'generated_text': ' to encourage good use of the same product\n\n(2)Where there is a strong preference among experts in'},
 {'generated_text': ", including the EU's 'Duke of Limbo' regulations as a 'guarantee that the countries"},
 {'generated_text': ', one based on international practice of the EU.\n\n1. A national harmonised standard: whether under'},
 {'generated_text': '. I find that most of the information I have for a particular type of document has nothing to do with the'}]

# Better LLM

In [35]:
from langchain_google_genai import ChatGoogleGenerativeAI
from IPython.display import Markdown

from dotenv import load_dotenv
load_dotenv()

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
result = llm.invoke("What is an LLM?")

print(result.__dict__.keys())
Markdown(result.content)

dict_keys(['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'example', 'tool_calls', 'invalid_tool_calls', 'usage_metadata'])


LLM stands for **Large Language Model**. It's a type of artificial intelligence (AI) that's specifically designed to understand and generate human-like text. 

Here's a breakdown of what makes LLMs special:

* **Massive Datasets:** LLMs are trained on huge amounts of text data, like books, articles, code, and websites. This allows them to learn the nuances of language and how words are used in different contexts.
* **Deep Learning:** LLMs use deep learning algorithms, which are complex mathematical models inspired by the human brain. These algorithms allow them to learn patterns and relationships within the data, enabling them to generate coherent and contextually relevant text.
* **Natural Language Processing (NLP):** LLMs are a core component of NLP, which focuses on enabling computers to understand and interact with human language. They are used in various NLP tasks, such as text summarization, translation, question answering, and chatbot development.

**Here are some examples of LLMs:**

* **GPT-3 (Generative Pre-trained Transformer 3):** Developed by OpenAI, GPT-3 is one of the most powerful LLMs available. It can generate creative content, translate languages, write different kinds of creative content, and answer your questions in an informative way.
* **LaMDA (Language Model for Dialogue Applications):** Developed by Google, LaMDA is designed specifically for conversational AI. It can engage in natural-sounding conversations, answering your questions and even holding opinions.
* **BERT (Bidirectional Encoder Representations from Transformers):** Developed by Google, BERT is particularly good at understanding the context of words within a sentence. It's widely used in NLP tasks like sentiment analysis and question answering.

**LLMs are rapidly evolving and have the potential to revolutionize many industries, including:**

* **Content Creation:** Generating articles, stories, poems, and even code.
* **Customer Service:** Providing automated support through chatbots.
* **Education:** Personalizing learning experiences and providing interactive tutoring.
* **Research:** Analyzing large datasets and generating reports.

However, LLMs also raise ethical concerns, such as the potential for bias, misinformation, and job displacement. It's important to use LLMs responsibly and to be aware of their limitations.


In [36]:
result_gpt2 = gpt2.invoke("What is an LLM?")
Markdown(result_gpt2)



To be precise, a LLM should be a type that defines a set of parameters. The first argument to an implementation of an LLM is the form of the argument. The second argument is the argument's definition, the final form of the argument, and so on. These are called a "argument semantics". These semantics are also often referred to as the "argument model". It's the same concept as for types, and will hold even with some generalizations based on LLM's semantics. LLM's semantics are implemented in a number of different ways, and some examples are given in the following section.

Basic LLMs

First, make sure that the argument's definition has already been applied at initialization, otherwise LLM will call the type again without an argument in place. However, sometimes this can occur. For example, when an integer type is passed to a function, the type of the method returned is not guaranteed to come from a valid function. In this case you are forced to convert the name of the method to a valid function call. Alternatively, in some cases, you might omit the name of the method, so that the resulting type will resemble the class definition.

Next, simply write the definition into the arguments, and then write the same for each argument as indicated in the LLM syntax.

This is a few examples. The final form may be a list. Note, that every argument declaration has a certain format.

If you include this version of a definition in your documentation, the compiler ignores all occurrences of the final version of the definition. This could cause you to break the grammar.

If you specify you believe that this example is the first time a function is provided, this is the first time you'll ever use the "f" (for --) notation to evaluate a call in the definition.

Other types may need to be specified and may also need to be called separately. This is where two values are taken off-line, and the "name" of the other is also taken off-line.

The "type" of the rest of an input argument is usually taken from the form of a name of the type. In this example, the name of the type is not taken into account.

The actual name of the variable, as defined, is given as a single list of "types".

To call a method, you also need to specify a specific argument; typically what the method calls, the call context, and all the parameters you call it with. For example, the following might be interpreted as follows:

return {... }

This "is" option will call a method that gets a value returned.

For more examples of how to write an LLM, see LLM vs. Type. In the next section, I'll present the problem with using the Type model and other classes (called "class constructors", or "interface classes") in class evaluation, in contrast to classes which are described below.

The LLM model is essentially the idea behind all of this documentation. This particular article will take a look at the LLM model described below by the name of a variable, the class, and the name of the methods and arguments.

LLLM Models

LLLM also refers to other types. For example, here is the LLM syntax for a method on some object.

type F = f a b... object F.prototype : (f: F) -> f... F.prototype : (f: F) -> f (object F)

The name, type, and constructor parameters may be assigned when defining an instance(or the object instance if not a constructor or an instanceclass). This is useful for defining generic methods to be used in an application.

This way: an instance function can be passed as an argument. If an instance variable is passed to f(foo), the given function will be called by doing something like this:

let foo = new Foo // foo is a constructor. let f a = new Foo // foo is a supertype (class F);

For example, you may want to take a copy argument to f(foo); and define the following:

let f = new Foo // foo is a supertype (class F);

The argument(s) function as indicated by the type is not only called by f(foo)(), but also by (f f a ) which is then called by doing nothing like this:

let f = new Foo // foo is in a constructor. let f a = new Foo // foo is a supertype (class F);

The constructor, f'. This method calls an object instance of F and prints out the following:

let f = new Foo // foo is

## Generate

In [37]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. Mention in which pages the answer is found.

{context}

Question: {question}

Helpful Answer:"""
prompt = PromptTemplate.from_template(template)


example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

print(example_messages[0].content)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible. Mention in which pages the answer is found.

filler context

Question: filler question

Helpful Answer:


In [38]:

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import RetrievalQA

def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        page_number = doc.metadata["page"] + 1 
        content_with_page = f"Page {page_number}:\n{doc.page_content}"
        formatted_docs.append(content_with_page)
    return "\n\n".join(formatted_docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain_type_kwargs = {"prompt": prompt}
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

query = "Describe the use of harmonised standards"
Markdown(rag_chain.invoke(query))

Harmonized standards, published in the Official Journal of the European Union, are presumed to meet the requirements of the regulation. This applies to quality management systems, risk management, and other aspects of the manufacturing process. (Page 16) Notified bodies are required to assess conformity with harmonized standards related to quality management systems. (Page 197) 


In [39]:
result = qa_chain.invoke({"query": query})
result

{'query': 'Describe the use of harmonised standards',
 'result': 'Harmonized standards, whose references have been published in the Official Journal of the European Union, are presumed to meet the requirements of the Regulation. This applies to both product requirements and system or process requirements. (Pages 1 and 2) \n',
 'source_documents': [Document(metadata={'Author': 'Publications Office', 'CreationDate': "D:20240724041003-07'00'", 'Creator': 'Arbortext Advanced Print Publisher 10.0.1465/W Unicode', 'ModDate': "D:20240808025522+02'00'", 'Producer': '3-Heights(TM) PDF to PDF-A Converter Shell 4.7.24.2 (http://www.pdf-tools.com)', 'Subject': ' ', 'Title': 'CL2017R0745EN0040010.0001.3bi_cp 1..1', 'file_path': 'document.pdf', 'page': 15, 'source': 'document.pdf', 'start_index': 1522, 'total_pages': 232}, page_content='which have been published in the Official Journal of the European\nUnion, shall be presumed to be in conformity with the requirements\nof this Regulation covered by 

In [40]:
Markdown(rag_chain.invoke("What is an LLM?"))

I'm sorry, but the provided text does not contain any information about "LLM." 


In [41]:
import ipywidgets as widgets
from IPython.display import display

# Function that processes the user's question
def process_input(user_input):
    complete_prompt = f"WIP"
    answer = rag_chain.invoke(user_input)
    return answer, complete_prompt

# Text input widget (for user question)
text_input = widgets.Text(
    description='Input:',
    placeholder='Type something here...',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Primary output widget to display LLM's answer
primary_output = widgets.Output()

# Secondary output widget for the complete generated prompt (collapsible)
secondary_output = widgets.Output()

# Progress indicator (shown while processing)
progress_indicator = widgets.Output()

# Event handler for when Enter is pressed in the text input
def on_text_submit(change):
    if change['type'] == 'change' and change['name'] == 'value':
        input_value = change.new.strip()
        if input_value == "":
            return

        # Show the progress indicator
        with progress_indicator:
            progress_indicator.clear_output()
            print("Processing... Please wait.")

        answer, complete_prompt = process_input(input_value)  # Processes the input

        with primary_output:
            primary_output.clear_output()       # Clears the previous output
            print(f"User Question: {input_value}")
            print("LLM Answer:")
            print(answer)

        with secondary_output:
            secondary_output.clear_output()     # Clears the previous output
            print("Complete prompt:")
            print(complete_prompt)

        # Hide the progress indicator after processing is complete
        with progress_indicator:
            progress_indicator.clear_output()

        change.new = ""

# Attach the event handler to the text input widget for Enter key submission
text_input.continuous_update = False
text_input.observe(on_text_submit, names='value', type="change")

# Make the complete generated prompt collapsible
accordion = widgets.Accordion(children=[secondary_output])
accordion.set_title(0, 'Complete generated prompt')

# Display all the fields: text input, LLM's answer, complete prompt
display(text_input, primary_output, accordion, progress_indicator)

Text(value='', continuous_update=False, description='Input:', layout=Layout(width='500px'), placeholder='Type …

Output()

Accordion(children=(Output(),), titles=('Complete generated prompt',))

Output()